<a href="https://colab.research.google.com/github/satani99/practical_deep_learning_for_coders/blob/main/fast_ai_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai.vision.all import *
path = untar_data(URLs.IMAGENETTE)

In [2]:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = dblock.dataloaders(path, bs=64)

In [3]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.672794,2.031946,0.415982,02:45
1,1.200496,1.871692,0.477969,02:43
2,0.952586,1.148194,0.658327,02:43
3,0.748588,0.712984,0.767364,02:43
4,0.577096,0.573428,0.823749,02:42


In [4]:
x, y = dls.one_batch()
x.mean(dim=[0,2,3]), x.std(dim=[0,2,3])

(TensorImage([0.4624, 0.4536, 0.4292], device='cuda:0'),
 TensorImage([0.2727, 0.2692, 0.2942], device='cuda:0'))

In [5]:
def get_dls(bs, size):
  dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                     get_items=get_image_files,
                     get_y=parent_label,
                     item_tfms=Resize(460),
                     batch_tfms=[*aug_transforms(size=size, min_scale=0.75),
                                 Normalize.from_stats(*imagenet_stats)])
  return dblock.dataloaders(path, bs=bs)

dls = get_dls(64, 224)

x, y = dls.one_batch()
x.mean(dim=[0, 2, 3]), x.std(dim=[0,2,3])

(TensorImage([-0.1530, -0.0102,  0.0919], device='cuda:0'),
 TensorImage([1.2526, 1.2740, 1.3924], device='cuda:0'))

In [6]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.572595,2.643660,0.393204,02:42
1,1.226252,2.007008,0.418969,02:42
2,0.927257,0.912076,0.723301,02:41
3,0.726913,0.670980,0.793129,02:40
4,0.574725,0.576684,0.821509,02:42


In [7]:
dls = get_dls(128, 128)
learn = Learner(dls, xresnet50(), loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(4, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.882573,2.722857,0.403659,02:17
1,1.294682,1.355122,0.590739,02:13
2,0.954408,0.893001,0.718820,02:12
3,0.748580,0.657982,0.788275,02:11


In [8]:
learn.dls = get_dls(64, 224)
learn.fine_tune(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.819718,0.984118,0.690814,02:42


epoch,train_loss,valid_loss,accuracy,time
0,0.658473,0.741530,0.771471,02:43
1,0.667902,0.838622,0.748320,02:42
2,0.579064,0.537091,0.830471,02:43
3,0.483856,0.442050,0.860344,02:43
4,0.416244,0.417730,0.871172,02:43


In [9]:
preds, targs = learn.tta()
accuracy(preds, targs).item()

0.8823748826980591

In [10]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(),
                metrics=accuracy, cbs=MixUp)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.162469,2.369188,0.399552,02:12
1,1.721825,2.414115,0.360344,02:12
2,1.481191,1.101809,0.671397,02:13
3,1.311470,0.801571,0.762883,02:13
4,1.204095,0.708794,0.783794,02:13


In [11]:
model = xresnet50()
learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropy(),
                metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.760962,2.830443,0.428305,02:12
1,2.255949,2.017212,0.658327,02:13
2,1.939927,1.856609,0.716580,02:13
3,1.752698,1.711360,0.757655,02:12
4,1.634320,1.602945,0.808439,02:13
